In [1]:
import os
import pandas as pd
import datasets
import torch

from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling



#для создания даталодера и датасета на торч
#from torch.utils.data import Dataset as torch_Dataset
#from torch.utils.data import DataLoader as torch_DataLoader

#для работы с датасетом
#from datasets import Dataset, DatasetDict
#from datasets import Image as Image_dataset
#трансформеры
#from transformers import AutoProcessor, BlipForConditionalGeneration
#from transformers import BlipTextModel, AutoTokenizer

from matplotlib import pyplot as plt
from tqdm.auto import tqdm
from PIL import Image
from IPython.display import FileLink

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(device)

In [19]:
text = """Мороз и солнце; день чудесный!
Еще ты дремлешь, друг прелестный —
Пора, красавица, проснись:
Открой сомкнуты негой взоры
Навстречу северной Авроры,
Звездою севера явись!
Вечор, ты помнишь, вьюга злилась,
На мутном небе мгла носилась;
Луна, как бледное пятно,
Сквозь тучи мрачные желтела,
И ты печальная сидела —
А нынче... погляди в окно:
Под голубыми небесами
Великолепными коврами,
Блестя на солнце, снег лежит;
Прозрачный лес один чернеет,
И ель сквозь иней зеленеет,
И речка подо льдом блестит.
Вся комната янтарным блеском
Озарена. Веселым треском
Трещит затопленная печь.
Приятно думать у лежанки.
Но знаешь: не велеть ли в санки
Кобылку бурую запречь?
Скользя по утреннему снегу,
Друг милый, предадимся бегу
Нетерпеливого коня
И навестим поля пустые,
Леса, недавно столь густые,
И берег, милый для меня."""

In [20]:
# Сохраним обучающие данные в .txt файл 
train_path = 'train_dataset.txt'
with open(train_path, "w") as f:
    f.write(text)

In [12]:
train_path = '/kaggle/input/train-building-adress/train_dataset.txt'

In [17]:
train_dir = '/kaggle/working/'

In [22]:
# Создание датасета
train_dataset = TextDataset(tokenizer=tokenizer, file_path=train_path, block_size=64)
  
# Создание даталодера (нарезает текст на оптимальные по длине куски)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [23]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuned", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=300, # number of training epochs
    per_device_train_batch_size=16, # batch size for training
    per_device_eval_batch_size=16,  # batch size for evaluation
    warmup_steps=10,# number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=16, # to make "virtual" batch size larger
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

In [24]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 trainer.train()                                                                              │
│   2                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:1645 in train                    │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:2007 in _inner_training_loop     │
│                                                                                                  │
│   2004 │   │   │   │   │   │   scale_after = self.scaler.get_scale()                             │
│   2005 │   │   │   │   │   │   optimizer_was_run = scale_before <= scale_after                   │
│   2006 │   │   │   │   │   else:                                                                 │
│ ❱ 2007 │   │   │   │   │   │   self.optimizer.step()                                             │
│   2008 │   │   │   │   │   │   optimizer_was_run = not self.accelerator.optimizer_step_was_skip  │
│   2009 │   │   │   │   │                                                                         │
│   2010 │   │   │   │   │   if optimizer_was_run:                                                 │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/accelerate/optimizer.py:140 in step                      │
│                                                                                                  │
│   137 │   │   │   │   # If we reduced the loss scale, it means the optimizer step was skipped    │
│   138 │   │   │   │   self._is_overflow = scale_after < scale_before                             │
│   139 │   │   │   else:                                                                          │
│ ❱ 140 │   │   │   │   self.optimizer.step(closure)                                               │
│   141 │                                                                                          │
│   142 │   def _switch_parameters(self, parameters_map):                                          │
│   143 │   │   for param_group in self.optimizer.param_groups:                                    │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:69 in wrapper                │
│                                                                                                  │
│     66 │   │   │   │   instance = instance_ref()                                                 │
│     67 │   │   │   │   instance._step_count += 1           